## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [ ]:
import os
import keras
import sklearn
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [ ]:
train, test = keras.datasets.cifar10.load_data()

In [ ]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [ ]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [ ]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                      activation="relu", 
                      name="hidden_layer"+str(i+1))(input_layer)
            x = keras.layers.BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                      activation="relu", 
                      name="hidden_layer"+str(i+1))(x)
            x = keras.layers.BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [ ]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 20
BATCH_SIZE = 1024
MOMENTUM = 0.95
BESTONLY = [True, False]
WEIGHTONLY = [False, True]

In [ ]:
results = {}
loadedmodel = {}
for bo, wo in itertools.product(BESTONLY, WEIGHTONLY):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print('BESTONLY:', bo, 'WEIGHTONLY:', wo)
    model = build_mlp(input_shape=x_train.shape[1:])
    #model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model_ckpt = keras.callbacks.ModelCheckpoint(
        filepath="./Day086_saved_models/B-%s-W-%s.h5" % (str(bo),str(wo)), 
        monitor="val_loss", 
        save_best_only=bo, 
        save_weights_only=wo)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True, 
              callbacks=[model_ckpt])

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]

    # Load back
    exp_name_tag = "B-%s-W-%s" % (str(bo),str(wo))
    
    if wo == True:
        loadedmodel[exp_name_tag] = build_mlp(input_shape=x_train.shape[1:])
        loadedmodel[exp_name_tag].compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
        loadedmodel[exp_name_tag].load_weights("./Day086_saved_models/%s.h5" % exp_name_tag)
    else:
        loadedmodel[exp_name_tag] = keras.models.load_model("./Day086_saved_models/%s.h5" % exp_name_tag)
        
    loss_loadback, acc_loadback = loadedmodel[exp_name_tag].evaluate(x_test, y_test)

    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc,
                             'loss-loadback': loss_loadback,
                             'acc-loadback': acc_loadback}

In [ ]:
color_bar = ["r", "g", "b", "y", "m", "k"]
plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
    plt.hlines(y=results[cond]['loss-loadback'], xmin=0, xmax=len(train_loss), linestyles='--', color=color_bar[i])
plt.title("Loss")
#plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
    plt.hlines(y=results[cond]['acc-loadback'], xmin=0, xmax=len(train_loss), linestyles='--', color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)
print(y_pred)

model2 = build_mlp(input_shape=x_train.shape[1:])
model2.load_weights('./Day086_saved_models/BESTONLY-False-WEIGHTONLY-True.h5')
y_pred2 = model2.predict(x_test)
print(y_pred2)

if y_pred.all() == y_pred2.all():
    print('y_pred == y_pred2')